**Tools And Agents**

In [2]:
#arxiv-wikipedia inbuilt tools
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [3]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name


'wikipedia'

In [4]:
api_arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_arxiv_wrapper)
arxiv

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250))

In [18]:
#custom tool
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader=WebBaseLoader("https://docs.langchain.com/langsmith/home")
doc=loader.load()
document=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(doc)
embedding=HuggingFaceEmbeddings()
vectordb=FAISS.from_documents(document,embedding)
retriever=vectordb.as_retriever()



In [20]:
from langchain_core.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-overview","search information regrading lang smith")
retriever_tool.name

'langsmith-overview'

In [21]:
tools=[wiki,arxiv,retriever_tool] 
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\HANA\\Langchain\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 StructuredTool(name='langsmith-overview', description='search information regrading lang smith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x000002304A1CF560>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x000002304A1CF9C0>)]

In [22]:
#run all the tools with agent amd LLM models
import os
import openai
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()
api_key=os.getenv('GROQ_API_KEY')
openai.api_key=os.getenv('OPENAI_API_KEY')
llm=ChatGroq(groq_api_key=api_key,model_name="llama-3.3-70b-versatile")


In [23]:
#prompt Template
from langsmith import Client
client=Client(api_key=os.getenv('LANGCHAIN_API_KEY'))
prompt=client.pull_prompt("hwchase17/openai-functions-agent")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
# #Agent
# from langchain_classic.agents import create_openai_tools_agent
# agent=create_openai_tools_agent(llm,tools,prompt)
# agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [44]:

from langchain_classic.agents import initialize_agent,AgentType
search_agent=initialize_agent(tools,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,handling_parsing_errors=True,verbose=True)
search_agent

AgentExecutor(verbose=True, tags=['zero-shot-react-description'], agent=ZeroShotAgent(llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, template="Answer the following questions as best you can. You have access to the following tools:\n\nwikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith-overview(query: 'str', callbacks: 'Callbacks' = None) -> 'str | tuple[str, list[Document]]' - search information regrading lang smith\n\nUse the following format:\n\n

In [ ]:
#Agent Executor
# from langchain_classic.agents import AgentExecutor
# agent_execute=AgentExecutor(agent=agent,tools=tools,verbose=True)
# agent_execute




AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [36]:
search_agent.invoke({"input":"what is langsmith"})

{'input': 'what is langsmith',
 'output': 'Langsmith is a tool for developing, debugging, and deploying LLM applications, providing features such as observability, evaluation, deployment, and platform setup, and is framework agnostic.'}

In [37]:
search_agent.invoke({"input":"what is machine learning"})

{'input': 'what is machine learning',
 'output': 'Machine learning is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without being explicitly programmed. It combines computer science, statistics, and engineering to enable machines to learn from data and improve their performance on a task over time. Machine learning has various applications, including data science, official statistics, and more, and is a rapidly evolving field with new techniques and advancements being published regularly.'}

In [39]:
search_agent.invoke({"what is the paper 1706.03762 about"})

{'input': {'what is the paper 1706.03762 about'},
 'output': 'The paper 1706.03762 is titled "Attention Is All You Need" and is written by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, and Illia Polosukhin. The paper discusses sequence transduction models, but the exact details of the paper are not provided in the observation. However, based on the title and authors, it appears to be a well-known paper in the field of natural language processing and machine learning.'}